## Case: Discrete Time Problem

Consider quasi-steady state (to march one time step):
$$\frac{1}{dt} u - \frac{1}{dt} u_n - \mu \Delta u + b \cdot \nabla u + \sigma u = f$$

Let $u_n = 0$, then we get
$$\frac{1}{dt} u - \mu \Delta u + b \cdot \nabla u + \sigma u = f$$

Use the following to get an exact solution:
$$- \mu \Delta u + b \cdot \nabla u + (\sigma + \frac{1}{dt}) u = f$$


Observations:
* I didn't notice that my boundary condition started at the previous time value. So I've changed u_D to start at t=dt.

L2 = [0.00184787979202, 0.00105810225271,0.000276831534501,4.64777180384e-05,4.12684044107e-06,6.41631919236e-07];

H1 = [0.312366263529,0.332888814706,0.195881509539,0.0568651780157,0.0124115103279,0.00438898711996];


In [11]:
%matplotlib inline 
import matplotlib
from dolfin import *
from __future__ import print_function
import math as m
import sympy as sym
import csv

x, y, t = sym.symbols('x[0], x[1], t')
sigma = 1.0
mu = 10**(-5) 
b=as_vector([2.0, 3.0])

# Iliescu Exact Solution
c = 16.0*sym.sin(sym.pi*t)
h = x*(1-x)*y*(1-y)
g = 2*mu**(-0.5)*(0.25**2 - (x - 0.5)**2 - (y - 0.5)**2 )
ue = c*h*(0.5+sym.atan(g)/sym.pi)

dt = 0.01

ue = sym.simplify(ue)
u_code = sym.printing.ccode(ue)
u_code = u_code.replace('M_PI','DOLFIN_PI')

fe = - mu*(sym.diff(sym.diff(ue,x),x) + sym.diff(sym.diff(ue,y),y))
fe += b[0]*sym.diff(ue,x) + b[1]*sym.diff(ue,y)
fe += (sigma + 1/dt)*ue

f_code = sym.printing.ccode(fe)
f_code = f_code.replace('M_PI','DOLFIN_PI')

In [12]:
def compute_errors(u_e, u, t, mesh):
	L2n = errornorm(u_e, u, norm_type='L2', degree_rise=3, mesh=mesh)
	H1n = errornorm(u_e, u, norm_type='H1', degree_rise=3, mesh=mesh)
	errors = {'L2 norm': L2n, 'H1 norm': H1n}
	return 'L2, ' + str(L2n) +', H1, '+ str(H1n) +', t, '+ str(t) +'\n'

In [15]:
def evolve(nx, u_code, f_code, sigma, mu, velocity):
    degree = 2
    
    u_exact = Expression(u_code, degree = degree, t=0.0)
    
    f = Expression(f_code, degree = degree, t=dt)

    mesh = UnitSquareMesh(nx,nx)

    Q = FunctionSpace(mesh, "CG", degree)

    # Set up boundary condition
    u_D = Expression(u_exact.cppcode, degree = degree, t=dt)
    
    def boundary(x, on_boundary):
        return on_boundary
    
    bc = DirichletBC(Q, u_D, boundary)

    # Test and trial functions
    u, v = TrialFunction(Q), TestFunction(Q)
    u_n = Function(Q)
    u_ = Function(Q)

    # Galerkin variational problem
    F = mu*dot(grad(v), grad(u))*dx + v*dot(velocity, grad(u))*dx + (sigma+1.0/dt)*v*u*dx - f*v*dx

    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)

    # Assemble matrices
    A1 = assemble(a1)
    bc.apply(A1)

    # Create progress bar
    progress = Progress('Time-stepping')
    set_log_level(PROGRESS)

    # Step 1 Solve on Coarse Grid
    b = assemble(L)
    bc.apply(b)
    solve(A1, u_.vector(), b, 'gmres')
    
    u_exact.t = dt
    errors = compute_errors(u_exact, u_, 0, mesh)
    print(errors)
    
    out_file_uexact = File("results/evolve_steady_u_exact_"+str(nx)+".pvd") 
    ue = interpolate(u_exact, Q)
    out_file_uexact << ue
    out_file_ubar = File("results/"+str(mu)+"evolve_steady_u_"+str(nx)+".pvd") 
    out_file_ubar << u_

In [16]:
for nx in [25, 50, 100, 200, 400, 800]:
    print('nx = '+str(nx))
    evolve(nx, u_code, f_code, sigma, mu, b)

nx = 25


--- Instant: compiling ---


Calling FFC just-in-time (JIT) compiler, this may take some time.
L2, 0.00184787979202, H1, 0.312366263529, t, 0

nx = 50
L2, 0.00105810225271, H1, 0.332888814706, t, 0

nx = 100
L2, 0.000276831534501, H1, 0.195881509539, t, 0

nx = 200
L2, 4.64777180384e-05, H1, 0.0568651780157, t, 0

nx = 400
L2, 4.12684044107e-06, H1, 0.0124115103279, t, 0

nx = 800
L2, 6.41631919236e-07, H1, 0.00438898711996, t, 0

